# Coarse Graining with flowerMD

## Overview:

Coarse graining in molecular dynamics (MD) simplifies complex systems by grouping atoms or molecules into beads, enhancing computational efficiency and enabling simulations on extended time scales.

The flowerMD package is designed in a way that simplifies the execution of MD simulations for coarse-grained systems. This is achieved by incorporating built-in functionalities that can transform an all-atom system into coarse-grained beads and define the force interactions between these beads, all with minimal user-side coding effort.

In this tutorial, we will go through the steps required to convert an all-atom strucutre to a coarse grained one and initialize a simulation for the modified system.

In [33]:
import warnings

warnings.filterwarnings("ignore")

In [34]:
from flowermd.base import Pack, Simulation
from flowermd.library import PPS, BeadSpring

### Step 1: Coarse graining an all-atom molecule

First, we initialize 300 atomistic PPS polymers and use the coarse graining method in `Molecule` to group atoms in one PPS molecule into a single bead.

In [35]:
pps_mol = PPS(num_mols=150, lengths=12)

First, let's examine the atomistic visualization of one of the PPS chains.

In [36]:
pps_mol.molecules[0].visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The `coarse_grain` method invokes the `GRiTS` coarse graining toolkit in the background. The `GRiTS` package, when provided with the SMILES notations of the beads, scans the atomic structure and automatically substitutes groups of atoms that match the SMILES pattern with a respective beads. 

In our PPS example, the `beads` parameter consists of a dictionary mapping arbitrary bead names (for example, here we named the bead "A") to their corresponding SMILES strings. Note that a strucuture can be coarse grained into multiple types of beads. Please see the example at the end of this tutorial for clarrification. 

In [37]:
pps_mol.coarse_grain(beads={"A": "c1ccc(S)cc1"})

**Important note regarding `coarse_grain`**: The coarse-grain method performs in-place alterations, resulting in the transformation of `pps_mol` from an all-atom structure to one where all particles are substituted with corresponding coarse-grained beads

As you can see in the visualization below, all the individual monomers have been substituted with "A" beads

In [38]:
pps_mol._cg_molecules[0].visualize(show_atomistic=True)

KeyboardInterrupt: 

We can visualize both the original atomistic structure and the new coarse-grained molecule.

In [7]:
pps_mol.molecules[0].visualize(show_atomistic=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Step 2: Define forces

To define the bonded and non-bonded interactions among the coarse beads, we utiize the `BeadSpring` class from the flowerMD libraray. This class generates a list of HOOMD force objects representing the Lennard-Jones (LJ) forces for the non-bonded interactions and harmonic bond forces, harmonic angle forces and periodic dihedral forces for the bonded interactions.

For each of these forces, specific parameters must be provided. Fpr instance, the LJ force requires `epsilon` and `sigma` parameters.
Please check [`HOOMD-Blue` documentation](https://hoomd-blue.readthedocs.io/en/stable/module-md-bond.html) for detailed information about the parameters required for [harmonic bonds](https://hoomd-blue.readthedocs.io/en/v4.1.0/module-md-bond.html), [harmonic angles](https://hoomd-blue.readthedocs.io/en/stable/module-md-angle.html) and [periodic rdihedrals](https://hoomd-blue.readthedocs.io/en/stable/module-md-dihedral.html).  

It's worth noting that the `BeadSpring` class serves as an illustrative example, generating forces based on commonly used force types. However, users have the flexibility to implement their own methods or classes to generate HOOMD force objects tailored to their specific requirements.

In [8]:
ff = BeadSpring(
    r_cut=2.5,
    beads={
        "A": dict(epsilon=1, sigma=0.2),
    },
    bonds={
        "A-A": dict(r0=0.64, k=500),
    },
    angles={"A-A-A": dict(t0=2.8, k=50)},
)

In [9]:
ff.hoomd_forces

### Step 3: Create the system

Using the `Pack` class, we pack a box of coarse grained chains. Note that, since the list of HOOMD force objects for the system has already been defined, there is no requirement to pass the `forcefield` parameter when configuring the system.

In [10]:
cg_system = Pack(molecules=pps_mol, density=0.5, edge=2, overlap=2)

In [11]:
cg_system.system.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Step 4: Start the simulation

Similar to initializing an atomisitc simulation, we pass in the `hoomd_snapshot` generated by the system along with the force objects generated by the `BeadSpring` class to start a simulation.

In [12]:
cg_sim = Simulation(
    initial_state=cg_system.hoomd_snapshot,
    forcefield=ff.hoomd_forces,
    gsd_write_freq=int(2e5 / 10),
)

Initializing simulation state from a gsd.hoomd.Frame.


In [13]:
cg_system.hoomd_snapshot.particles.types

['A']

In [14]:
cg_sim.run_NVT(duration=2e5, temperature=3.0, tau_kt=1.0)

Step 10500 of 200000; TPS: 5018.28; ETA: 0.6 minutes
Step 21000 of 200000; TPS: 5141.77; ETA: 0.6 minutes
Step 31500 of 200000; TPS: 5151.59; ETA: 0.5 minutes
Step 42000 of 200000; TPS: 5129.96; ETA: 0.5 minutes
Step 52500 of 200000; TPS: 5114.8; ETA: 0.5 minutes
Step 63000 of 200000; TPS: 5107.63; ETA: 0.4 minutes
Step 73500 of 200000; TPS: 5102.79; ETA: 0.4 minutes
Step 84000 of 200000; TPS: 5094.64; ETA: 0.4 minutes
Step 94500 of 200000; TPS: 5089.08; ETA: 0.3 minutes
Step 105000 of 200000; TPS: 5082.07; ETA: 0.3 minutes
Step 115500 of 200000; TPS: 5078.93; ETA: 0.3 minutes
Step 126000 of 200000; TPS: 5072.38; ETA: 0.2 minutes
Step 136500 of 200000; TPS: 5068.13; ETA: 0.2 minutes
Step 147000 of 200000; TPS: 5062.82; ETA: 0.2 minutes
Step 157500 of 200000; TPS: 5057.28; ETA: 0.1 minutes
Step 168000 of 200000; TPS: 5054.44; ETA: 0.1 minutes
Step 178500 of 200000; TPS: 5051.86; ETA: 0.1 minutes
Step 189000 of 200000; TPS: 5050.5; ETA: 0.0 minutes
Step 199500 of 200000; TPS: 5048.61; ET

In [15]:
cg_sim.flush_writers()

In [ ]:
from cmeutils.visualize import FresnelGSD

sim_visualizer = FresnelGSD(
    gsd_file="trajectory.gsd", frame=-1, view_axis=(1, 1, 1)
)
sim_visualizer.view()

In [ ]:
sim_visualizer.height = 20

In [ ]:
sim_visualizer.view()